Kim, M. J., Pertsch, K., Karamcheti, S., Xiao, T., Balakrishna, A., Nair, S., ... & Finn, C. (2024). [OpenVLA: An Open-Source Vision-Language-Action Model](https://arxiv.org/abs/2406.09246). arXiv preprint arXiv:2406.09246.

**Abstract**

<!-- Large policies pretrained on a combination of Internet-scale vision-language data and diverse robot demonstrations have the potential to change how we teach robots new skills: rather than training new behaviors from scratch, we can fine-tune such vision-language-action (VLA) models to obtain robust, generalizable policies for visuomotor control. Yet, widespread adoption of VLAs for robotics has been challenging as 1) existing VLAs are largely closed and inaccessible to the public, and 2) prior work fails to explore methods for efficiently fine-tuning VLAs for new tasks, a key component for adoption. Addressing these challenges, we introduce OpenVLA, a 7B-parameter open-source VLA trained on a diverse collection of 970k real-world robot demonstrations. OpenVLA builds on a Llama 2 language model combined with a visual encoder that fuses pretrained features from DINOv2 and SigLIP. As a product of the added data diversity and new model components, OpenVLA demonstrates strong results for generalist manipulation, outperforming closed models such as RT-2-X (55B) by 16.5% in absolute task success rate across 29 tasks and multiple robot embodiments, with 7x fewer parameters. We further show that we can effectively fine-tune OpenVLA for new settings, with especially strong generalization results in multi-task environments involving multiple objects and strong language grounding abilities, and outperform expressive from-scratch imitation learning methods such as Diffusion Policy by 20.4%. We also explore compute efficiency; as a separate contribution, we show that OpenVLA can be fine-tuned on consumer GPUs via modern low-rank adaptation methods and served efficiently via quantization without a hit to downstream success rate. Finally, we release model checkpoints, fine-tuning notebooks, and our PyTorch codebase with built-in support for training VLAs at scale on Open X-Embodiment datasets. -->
在互联网规模的视觉-语言数据和多样化机器人演示的组合上预训练的大型策略, 有潜力改变我们教授机器人新技能的方式: 我们可以微调这种视觉-语言-动作 (VLA) 模型, 以获得鲁棒、可泛化的视觉运动控制策略, 而不是从头开始训练新行为。然而, 将 VLA 广泛用于机器人一直具有挑战性, 因为: 1) 现有的 VLA 大多是封闭的, 不向公众开放, 2) 先前的工作未能探索有效微调 VLAs 的方法, 用于新任务, 这是采用的关键组件。为应对这些挑战, 我们提出 OpenVLA, 一个 7B 参数的开源 VLA, 它在 970k 个现实世界机器人演示的多样化集合上进行训练。OpenVLA 建立在 Llama 2 语言模型上, 结合了一个视觉编码器, 该视觉编码器融合了来自 DINOv2 和 SigLIP 的预训练特征。作为增加数据多样性和新的模型组件的产物, OpenVLA 在通用操作方面展现处强有力的结果, 在 29 项任务和多个机器人实例中的绝对任务成功率方面, 比 RT-2-X (55B) 等封闭模型高 16.5%, 参数减少 7 倍。我们进一步表明, 我们可以有效地微调 OpenVLA 用于新配置, 在多任务环境(包含多对象和强大的语言基础能力)中获得特别强大的泛化结果, 并且比富有表现力的从头开始的模仿学习方法(例如 Diffusion Policy)高 20.4%。我们还探索了计算效率; 作为一项单独的贡献, 我们表明 OpenVLA 可以通过流行的低秩自适应方法在消费者 GPUs 上进行微调, 并通过量化高效地提供服务, 而不会影响下游成功率。最后，我们发布了模型检查点、微调笔记和我们的 PyTorch 代码库, 内置支持在 Open X-Embodiment 数据集上大规模训练 VLAs。


# Introduction
<!-- A key weakness of learned policies for robotic manipulation is their inability to generalize beyond their training data: while existing policies trained for individual skills or language instructions have the capacity to extrapolate behaviors to new initial conditions such as object positions or lighting [2, 3], they lack robustness to scene distractors or novel objects [4, 5] and struggle to execute unseen task instructions [6, 7]. Yet beyond robotics, existing foundation models for vision and language such as CLIP [8], SigLIP [9], and Llama 2 [10] are capable of these types of generalization and more, stemming from the priors captured by their Internet-scale pretraining datasets. While reproducing this scale of pretraining for robotics is still an open challenge — even the largest robot manipulation datasets [1, 11] only have 100K to 1M examples – this imbalance suggests an opportunity: using existing foundation models for vision and language as a core building block for training robotic policies that can generalize to objects, scenes, and tasks beyond their training data. -->
机器人操作学习策略的一个关键弱点是无法在训练数据之外进行泛化: 尽管现有策略(被训练用于个别技能或语言指令)能够将行为推断到新的初始条件(例如物体位置或光照 [2, 3]), 但它们对场景干扰物或新物体缺乏鲁棒性 [4, 5], 并且难以执行未见过的任务指令 [6, 7]。然而, 除了机器人技术之外, 现有的视觉和语言基础模型(如 CLIP [8]、SigLIP [9] 和 Llama 2 [10])能够实现这些类型的泛化, 甚至更多, 源于其互联网规模预训练数据集捕获的先验。虽然复现这种规模的预训练用于机器人仍然是一个开放的挑战——即使最大的机器人操作数据集 [1, 11] 也只有 100K 到 1M 示例——但这种不平衡意味着机会: 使用现有的视觉和语言基础模型作为训练机器人策略的核心构建块, 这些策略可以泛化到训练数据之外的对象、场景和任务。

Towards this goal, existing work has explored integrating pretrained language and vision-language models for robotic representation learning [12–14] and as a component in modular systems for task planning and execution [15, 16]. More recently, they have been used for directly learning vision-language-action models [VLAs; 1, 7, 17, 18] for control. VLAs provide a direct instantiation of using pretrained vision-and-language foundation models for robotics, directly fine-tuning visually-conditioned language models (VLMs) such as PaLI [19, 20] to generate robot control actions. By building off of strong foundation models trained on Internet-scale data, VLAs such as RT-2 [7] demonstrate impressive robustness results, as well as an ability to generalize to novel objects and tasks, setting a new standard for generalist robot policies. Yet, there are two key reasons preventing the widespread use of existing VLAs: 1) current models [1, 7, 17, 18] are closed, with limited visibility into model architecture, training procedures, and data mixture, and 2) existing works do not provide best practices for deploying and adapting VLAs to new robots, environments, and tasks — especially on commodity hardware (e.g., consumer-grade GPUs). We argue that to develop a rich foundation for future research and development, robotics needs open-source, generalist VLAs that support effective fine-tuning and adaptation, akin to the existing ecosystem around open-source language models [21–24].
为了实现这一目标, 现有的研究已经探索了集成预训练的语言和视觉-语言模型, 用于机器人表示学习 [12–14], 并作为模块化系统的组件, 用于任务规划和执行 [15, 16]。最近, 它们已被用于直接学习视觉-语言-动作模型 [VLAs; 1, 7, 17, 18], 用于控制。VLAs 为机器人提供了使用预训练的视觉和语言基础模型的直接实例, 直接微调视觉条件语言模型 (VLMs)(如 PaLI [19, 20]), 以生成机器人控制动作。通过建立(在互联网规模数据上训练过的)强大的基础模型, RT-2 [7] 等 VLAs 展示了令人印象深刻的鲁棒结果, 以及泛化到新对象和新任务的能力, 为通用机器人策略设定了新标准。然而, 有两个主要原因, 阻碍了现有 VLAs 的广泛使用: 1) 当前模型 [1, 7, 17, 18] 是封闭的, 对模型架构、训练程序和数据混合的有限可见性,

2) 现有工作没有提供将 VLA 部署和适应新机器人、环境和任务的最佳实践——尤其是在商品硬件（例如消费级 GPU）上。

我们认为，要为未来的研究和开发奠定坚实的基础，机器人技术需要开源、通用的 VLA，以支持有效的微调和适应，类似于围绕开源语言模型的现有生态系统 [21–24]。

To this end, we introduce OpenVLA, a 7B-parameter open-source VLA that establishes a new state of the art for generalist robot manipulation policies.1 OpenVLA consists of a pretrained visually-conditioned language model backbone that captures visual features at multiple granularities, fine-tuned on a large, diverse dataset of 970k robot manipulation trajectories from the Open-X Embodiment [1] dataset—a dataset that spans a wide range of robot embodiments, tasks, and scenes. As a product of increased data diversity and new model components, OpenVLA outperforms the 55B-parameter RT-2-X model [1, 7], the prior state-of-the-art VLA, by 16.5% absolute success rate across 29 evaluation tasks on the WidowX and Google Robot embodiments. We additionally investigate efficient fine-tuning strategies for VLAs, a new contribution not explored in prior work, across 7 diverse manipulation tasks spanning behaviors from object pick-and-place to cleaning a table. We find that fine-tuned OpenVLA policies clearly outperform fine-tuned pretrained policies such as Octo [5]. Compared to from-scratch imitation learning with diffusion policies [3], fine-tuned OpenVLA shows substantial improvement on tasks involving grounding language to behavior in multi-task settings with multiple objects. Following these results, we are the first to demonstrate the effectiveness of compute-efficient fine-tuning methods leveraging low-rank adaptation [LoRA; 26] and model quantization [27] to facilitate adapting OpenVLA models on consumer-grade GPUs instead of large server nodes without compromising performance. As a final contribution, we open-source all models, deployment and fine-tuning notebooks, and the OpenVLA codebase for training VLAs at scale, with the hope that these resources enable future work exploring and adapting VLAs for robotics.

# Related Work
**Visually-Conditioned Language Models** Visually-conditioned language models (VLMs), which are trained on Internet-scale data to generate natural language from input image(s) and language prompts, have been adopted for myriad applications from visual question answering [28–31] to object localization [32, 33]. One of the key advances fueling recent VLMs are model architectures that bridge features from pretrained vision encoders [8, 9, 25] with pretrained language models [10, 23, 34–36], directly building on advances in both computer vision and natural language modelling to create powerful multimodal models. While early work explored various architectures for cross-attending between vision and language features [37–41], new open-source VLMs [20, 42–44] have converged on a simpler "patch-as-token" approach, in which patch features from pretrained visual transformers are treated as tokens, and are then projected into the input space of a language model. This simplicity makes it easy to repurpose existing tools for training language models at scale for VLM training. We employ these tools in our work to scale VLA training, and specifically use VLMs from Karamcheti et al. [44] as our pretrained backbone, as they are trained from multi-resolution visual features, fusing low-level spatial information from DINOv2 [25] with higher-level semantics from SigLIP [9] to aid in visual generalization.


**Generalist Robot Policies** A recent trend in robotics works towards training multi-task "generalist" robot policies [2, 6, 45–49] on large diverse robot datasets [1, 2, 6, 11, 45, 49–56], spanning many different robot embodiments [1, 5, 53, 57–66]. Notably, Octo [5] trains a generalist policy that can control multiple robots out-of-the-box and allows for flexible fine-tuning to new robot setups. A key difference between these approaches and OpenVLA is the model architecture. Prior works like Octo typically compose pretrained components such as language embeddings or visual encoders with additional model components initialized from scratch [2, 5, 6], learning to “stitch” them together during the course of policy training. Unlike these works, OpenVLA adopts a more end-to-end approach, directly fine-tuning VLMs to generate robot actions by treating them as tokens in the language model vocabulary. Our experimental evaluation shows that this simple yet scalable pipeline substantially boosts performance and generalization ability over prior generalist policies.


Vision-Language-Action Models A number of works have explored the use of VLMs for robotics, e.g., for visual state representations [12, 13], object detection [67], high-level planning [16], and for providing a feedback signal [68–71]. Others integrate VLMs directly into end-to-end visuomotor manipulation policies [14, 15], but incorporate significant structure into the policy architecture or require calibrated cameras, which limits their applicability. A number of recent works have explored similar recipes to ours and directly fine-tuned large pretrained VLMs for predicting robot actions [1, 7, 17, 18, 72]. Such models are often referred to as vision-language-action models (VLAs), since they fuse robot control actions directly into VLM backbones. This has three key benefits: (1) it performs alignment of pretrained vision and language components on a large, Internet-scale vision-language dataset, (2) the use of a generic architecture, not custom-made for robot control, allows us to leverage the scalable infrastructure underlying modern VLM training [73–75] and scale to training billion-parameter policies with minimal code modifications, and (3) it provides a direct pathway for robotics to benefit from the rapid improvements in VLMs. Existing works on VLAs either focus on training and evaluating in single robot or simulated setups [72, 76] and thus lack generality, or are closed and do not support efficient fine-tuning to new robot setups [1, 7, 17, 18]. Most closely related, RT-2-X [1] trains a 55B-parameter VLA policy on the Open X-Embodiment dataset and demonstrates state-of-the-art generalist manipulation policy performance. However, our work differs from RT-2-X in multiple important aspects: (1) by combining a strong open VLM backbone with a richer robot pretraining dataset, OpenVLA outperforms RT-2-X in our experiments while being an order of magnitude smaller; (2) we thoroughly investigate fine-tuning of OpenVLA models to new target setups, while RT-2-X does not investigate the fine-tuning setting; (3) we are the first to demonstrate the effectiveness of modern parameter-efficient fine-tuning and quantization approaches for VLAs; and (4) OpenVLA is the first generalist VLA that is open-source and thus supports future research on VLA training, data mixtures, objectives, and inference.